# Nomer 1 : Pre Processing Data, Tfidf dan K-Means

<h2>Import Library</h2>

<p>Langkah pertama adalah melakukan import library yang dimana nantinya digunakan untuk membaca data mauapaun memanipulasi data, untuk tahapan preprocessing, Tf-idf, maupun untuk K means clustering</p>

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
%matplotlib inline

<h2>Membaca Dokumen</h2>

<p>Setelahnya, proses yang dilakukan adalah membuka dokumen hasil Crawling yang telah dilakukan sebelumnya.</p>
<p>Juga, menampilkan file hasil crawling, dan dapat dilihat dibawah data hasil crawling berjumlah 15</p>

In [2]:
data = pd.read_csv(r'hasil_crawling.csv')
data

,Judul,Abstrak
0,Pengaruh Retailing Mix terhadap Keputusan Pemb...,Tujuan penelitian ini adalah untuk mengetahui ...
1,"PENGARUH PERSEPSI MANFAAT, PERSEPSI KEMUDAHAN,...",Tujuan penelitian ini adalah untuk mengidentif...
2,PELAPORAN BIAYA KUALITAS UNTUK MEMINIMALKAN TE...,ABSTRAK\r\n\r\nPenelitian ini mengungkapkan be...
3,PENGARUH KOMPETENSI DOSEN TERHADAP KINERJA DOS...,"Abstrak\r\n\r\nAththaariq, Pengaruh Kompetensi..."
4,PENGARUH PENGEMBANGAN KARIR TERHADAP SEMANGAT ...,PLN merupakan Badan Usaha Milik Negara yang be...
5,Analisis Inovasi Dan Keunggulan Bersaing \r\nD...,ABSTRAK\r\n\tTujuan penelitian ini adalah mend...
6,PENGARUH TIPE KEPEMIMPINAN TERHADAP PRESTASI K...,"ABSTRAK\r\n\tDharma Abidin Syah,Kesimpulan: (1..."
7,PENGARUH PERILAKU KONSUMEN TERHADAP KEPUTUSAN ...,"ABSTRAK\r\nHaryono Arifin, Pengaruh Perilaku K..."
8,PENGARUH DIMENSI KUALITAS PELAYANAN TERHADAP K...,ABSTRAK\r\n\r\nTujuan penelitian ini adalah un...
9,ANALISIS TINGKAT RISIKO KREDIT \r\nPADA PD. BP...,Hasil dari penelitian ini dari perhitungan Cre...


<p>Dikarenakan pada data ke 15 atau index 14 data Abstraknya NaN atau kosong, maka disini data pada baris tersebut akan dihilangkan</p>
<p>Juga disini akan membuang kolom yang tidak digunakan, kolom yang dibuang adalah kolom Judul.</p>

In [3]:
data.drop([14], axis=0, inplace=True)
data.drop(['Judul'],axis=1,inplace=True)
data

,Abstrak
0,Tujuan penelitian ini adalah untuk mengetahui ...
1,Tujuan penelitian ini adalah untuk mengidentif...
2,ABSTRAK\r\n\r\nPenelitian ini mengungkapkan be...
3,"Abstrak\r\n\r\nAththaariq, Pengaruh Kompetensi..."
4,PLN merupakan Badan Usaha Milik Negara yang be...
5,ABSTRAK\r\n\tTujuan penelitian ini adalah mend...
6,"ABSTRAK\r\n\tDharma Abidin Syah,Kesimpulan: (1..."
7,"ABSTRAK\r\nHaryono Arifin, Pengaruh Perilaku K..."
8,ABSTRAK\r\n\r\nTujuan penelitian ini adalah un...
9,Hasil dari penelitian ini dari perhitungan Cre...


<h2>Stopword</h2>
<p>Disini menggunakan stopwords indonesia, juga menambahkan beberapa stopword yang mungkin dibutuhkan untuk menghilangkan kata-kata yang tidak diperlukan.</p>

In [4]:
list_stopwords = stopwords.words('indonesian')


# manualy add stopword
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

<h2>Stemming dan Tokenizing</h2>

<p>Pada bagian ini, membuat fungsi untuk proses stemming dan tokensisasi sekaligus mengubah ukuran huruf menjadi lower case semua.</p>

In [5]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')

def tokenize(text):
    return [stemmer.stem(word) for word in tokenizer.tokenize(text.lower())]

<h2>Tfidf</h2>

<p>Pada prses ini dilakukan proses Tfidf</p>
<p>Dalam TfidfVectorizer sendiri telah terset default untuk membuat semua huruf menjadi lowercase </p>
</p>

<p>Untuk rumus Tfidf sendiri adalah sebagai berikut</p>

$w_{i, j}=t f_{i, j} \times \log \left(\frac{N}{d f_{i}}\right)$

$
\begin{aligned}
t f_{i, j} &=\text { banyaknya kata- } i \text { pada dokumen ke- } j \\
N &=\text { total dokumen } \\
d f_{i} &=\text { banyaknya dokumen yang mengandung kata ke- } i
\end{aligned}
$

In [6]:
desc = data['Abstrak'].values
vectorizer = TfidfVectorizer(stop_words = list_stopwords, tokenizer = tokenize)
X = vectorizer.fit_transform(desc)
word_features = vectorizer.get_feature_names() 

C:\Users\Arlaz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arti', 'asa', 'buah', 'bum', 'enak', 'hadap', 'ikan', 'ira', 'jenak', 'kali', 'kena', 'kerja', 'khusus', 'laku', 'langsung', 'lihat', 'maksud', 'masuk', 'mata', 'mena', 'nyata', 'olah', 'orang', 'rupa', 'sampa', 'sangkut', 'sekal', 'sekira', 'sin', 'tama', 'tuju', 'upa', 'utama'] not in stop_words.
  warnings.warn(
C:\Users\Arlaz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


<p>Dapat dilihat disini, jumlah fitur yang ada berjumlah 430 dan ditampilkan juga 50 fitur awal</p>

In [7]:
print(len(word_features))
print(word_features[:50])

430
['a', 'abidin', 'abstrak', 'accidental', 'adjusted', 'administrasi', 'agung', 'akademik', 'aksidental', 'aktif', 'akurat', 'alami', 'alat', 'alkirom', 'aman', 'ambil', 'analis', 'analisis', 'analitik', 'analysis', 'anova', 'anugrahini', 'aplikasi', 'apresisasikan', 'arifin', 'arti', 'association', 'asumsi', 'aththaariq', 'atmosfer', 'atribut', 'aulia', 'autoritarian', 'autoritariansecara', 'b', 'badan', 'bangkal', 'bank', 'banking', 'bantu', 'barokah', 'bebas', 'beda', 'beli', 'beta', 'biaya', 'bidang', 'bimbing', 'bisnis', 'bpr']


<h2>K Means Clustering</h2>

<p>Disini setelah melakukan proses Tfidf selanjutnya akan dilakukan pembagian fitur-fitur kedalam beberapa cluster sesuai yang diinginkan dengan menggunakan metode Kmeans.</p>

<h3>5 Clusters</h3>
<p>Disini akan dilakukan pembagian dari 5 cluster dan proses pembagiannya dilakukan dengan 20 iterasi. </p>
<p>Setiap clusternya akan ditampilkan 20 fitur.</p>

In [8]:
kmeans = KMeans(n_clusters = 5, n_init = 20)
kmeans.fit(X)
common_words = kmeans.cluster_centers_.argsort()[:,-1:-21:-1]
space = 0
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(word_features[word] for word in centroid))

0 : beli, putus, konsumen, variabel, uji, vario, honda, pengaruh, harga, merek, teliti, perilaku, psikologis, merchandise, mix, promosi, gerai, atmosfer, lokasi, hadap
1 : tingkat, biaya, lamongan, unggul, saing, inovasi, npl, resiko, bank, lapor, cacat, profitabilitas, alami, produk, kualitas, pasar, risiko, daerah, teliti, optik
2 : x, variabel, kerja, pengaruh, kompetensi, pegawai, hadap, pimpin, kembang, uji, prestasi, teliti, nilai, tipe, signifikan, kompensasi, produktivitas, y, pln, puas
3 : akademik, puas, portal, langgan, indeks, informasi, webqual, performance, administrasi, importance, mahasiswa, madura, universitas, layan, mutu, situs, website, utm, teknis, ki
4 : persepsi, dimensi, internet, banking, ulang, minat, bri, distro, brand, cranberries, association, kiddrock, mudah, teliti, sedia, variabel, pakai, langgan, metode, identifikasi


<p>Untuk hasil tanpa batasan penampilan tiap cluster hanya 20 akan menjadi seperti berikut</p>

In [9]:
kmeans = KMeans(n_clusters = 5, n_init = 20)
kmeans.fit(X)
common_words = kmeans.cluster_centers_.argsort()
space = 0
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(word_features[word] for word in centroid))

0 : a, pedagogik, pd, pasar, partisipasi, parsial, pangsa, pangarengan, pamekasan, paktek, pakai, orgnisasional, orang, optimalisasi, optik, omzet, olah, observasional, observasi, npl, pegawai, pegawaidikantor, pengaruh, pengembnagan, probality, primer, pribadi, prestasi, presepsi, prambon, praktek, positivisme, positif, nilai, populasi, pln, pimpin, persepsi, periodik, perinci, perilaku, performing, perangkat, pengolaha, pns, negeri, negara, nasional, x, loan, listrik, linier, likert, lihat, laut, latih, lapor, luas, lapang, landas, lamongan, laku, laksana, laissez, lai, kwanyar, kumpul, kuisioner, langsung, produk, lunak, maksimal, nasabah, muhammad, mudah, mt, ms, motor, motivasi, moch, mm, m, mix, minat, milik, metode, merek, merchandise, mentor, menganalisa, mart, manusia, minimal, kuesioner, produktivitas, profitabilitas, terap, terampil, tekan, tawar, tanggap, tabel, syarat, syah, surya, survei, sumbr, sumber, suasana, studi, status, statistik, st, square, spss, terima, thitung,

<h3> 10 Clusters</h3>
<p>Sama seperti saat 5 cluster, akan tetapi disini membagi menjadi 10 cluster</p>

In [10]:
kmeans = KMeans(n_clusters = 10, n_init = 20)
kmeans.fit(X)
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
#print(common_words)
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(word_features[word] for word in centroid))

0 : akademik, puas, portal, langgan, indeks, informasi, webqual, performance, administrasi, importance
1 : kompetensi, dosen, x, pedagogik, profesional, kerja, variabel, pengaruh, universitas, madura
2 : variabel, x, pimpin, tipe, pegawai, kompensasi, pengaruh, puas, hadap, konsumen
3 : inovasi, saing, unggul, pasar, reza, optik, lamongan, data, jawa, tingkat
4 : biaya, lapor, profitabilitas, cacat, kualitas, risiko, produk, tingkat, pengaruh, usaha
5 : persepsi, internet, banking, minat, ulang, bri, mudah, variabel, layan, kecuali
6 : dimensi, distro, brand, cranberries, kiddrock, association, persepsi, pakai, langgan, data
7 : beli, putus, konsumen, variabel, uji, vario, honda, pengaruh, harga, merek
8 : resiko, npl, bank, turun, risk, ratio, pd, bpr, credit, tingkat
9 : kembang, kerja, produktivitas, pln, faktor, latih, karyawan, semangat, karir, x
